## Load Dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/ming/Desktop/dsschool/titanic/train.csv", index_col="PassengerId")
print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
test = pd.read_csv("/Users/ming/Desktop/dsschool/titanic/test.csv", index_col="PassengerId")
print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing
Sex, Fare, Embarked 컬럼에 대해서

### 1. Sex

In [3]:
train.loc[train["Sex"] == "male", "Sex_encode"]=0
train.loc[train["Sex"] == "female", "Sex_encode"]=1

print(train.shape)
train[["Sex", "Sex_encode"]].head()

(891, 12)


,Sex,Sex_encode
PassengerId,,
1,male,0.0
2,female,1.0
3,female,1.0
4,female,1.0
5,male,0.0


In [4]:
test.loc[test["Sex"]=="male", "Sex_encode"]=0
test.loc[test["Sex"]=="female", "Sex_encode"]=1

print(test.shape)
test[["Sex", "Sex_encode"]].head()

(418, 11)


,Sex,Sex_encode
PassengerId,,
892,male,0.0
893,female,1.0
894,male,0.0
895,male,0.0
896,female,1.0


### 2. Fill in missing Fare

In [5]:
#채워넣기 전에 사본을 만들기 위해서 "Fare_fillin"으로 복사한 열을 먼저 만든다._ 원본 훼손 금지
#여기서 에러 났었음
train["Fare_fillin"] = train["Fare"]
test["Fare_fillin"] = test["Fare"]

print(test.shape)
test[["Fare", "Fare_fillin"]].head()

(418, 12)


,Fare,Fare_fillin
PassengerId,,
892,7.8292,7.8292
893,7.0000,7.0000
894,9.6875,9.6875
895,8.6625,8.6625
896,12.2875,12.2875


In [6]:
#fare 빈칸에 fillin 열에 0 채우기
test.loc[test["Fare"].isnull(), "Fare_fillin"] = 0

test[test["Fare"].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Fare_fillin
PassengerId,,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0.0,0.0


### 3. Encode Embarked

In [7]:
#간단하게, c ==0, s ==1, q ==2 이렇게 숫자로 인코딩하면, 나중에 논리에서 문제가 생김 2*s = q???
# 두개면 0,1로 가능

# one hot encoding
# c == [True,False,False]
# s == [False,True,False]
# q == [False,False,True]

# 판다스에서 컬럼을 추가할 때는 그 갯수만 맞으면 된다. 

In [8]:
# 질문_ 원래 아래와 같은 등호식에서 논리연산을 먼저하는 건가?? _ 그냥 상관이 없다고 함...ㅋㅋ
train["Embarked_C"] = train["Embarked"] == "C"
train["Embarked_S"] = train["Embarked"] == "S"
train["Embarked_Q"] = train["Embarked"] == "Q"

print(train.shape)
train[["Embarked_C", "Embarked_S", "Embarked_Q"]].head()

(891, 16)


,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,
1,False,True,False
2,True,False,False
3,False,True,False
4,False,True,False
5,False,True,False


In [9]:
test["Embarked_C"] = test["Embarked"] == "C"
test["Embarked_S"] = test["Embarked"] == "S"
test["Embarked_Q"] = test["Embarked"] == "Q"

print(test.shape)
test[["Embarked_C", "Embarked_S", "Embarked_Q"]].head()

(418, 15)


,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,
892,False,False,True
893,False,True,False
894,False,False,True
895,False,True,False
896,False,True,False


## Train
feature 와 label을 설정
feature에는 
label은 우리가 답으로 구하고자 하는 "Survived"에 해당

In [10]:
feature_name = ["Pclass","Sex_encode","Fare_fillin", "Embarked_C", "Embarked_S", "Embarked_Q"]

feature_name

['Pclass',
 'Sex_encode',
 'Fare_fillin',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q']

In [11]:
x_train = train[feature_name]

print(x_train.shape)
x_train.head()

(891, 6)


,Pclass,Sex_encode,Fare_fillin,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
1,3,0.0,7.2500,False,True,False
2,1,1.0,71.2833,True,False,False
3,3,1.0,7.9250,False,True,False
4,1,1.0,53.1000,False,True,False
5,3,0.0,8.0500,False,True,False


In [12]:
x_test = test[feature_name]

print(x_test.shape)
x_test.head()

(418, 6)


,Pclass,Sex_encode,Fare_fillin,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
892,3,0.0,7.8292,False,False,True
893,3,1.0,7.0000,False,True,False
894,2,0.0,9.6875,False,False,True
895,3,0.0,8.6625,False,True,False
896,3,1.0,12.2875,False,True,False


In [13]:
label_name = "Survived"

y_train = train[label_name]
print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

## Use Decision Tree

In [14]:
# sklearn.tree라는 모듈에서 DecisionTreeClassifier을 가져오겠다.
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 5)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
# 의사결정나무 쓰는 방법
# (1) fit_학습(train) -> (2) predict_예측(test)

model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
# 예측값을 predictions라는 변수에 넣는다.
# predictions에서는 head가 지원이 안된다. 
predictions = model.predict(x_test)
print(predictions.shape)
predictions[0:5]

(418,)


array([0, 0, 0, 0, 1])

## Submit

In [17]:
# 원래 줬던 예시 파일 출력
submit = pd.read_csv("/Users/ming/Desktop/dsschool/titanic/gender_submission.csv", index_col = "PassengerId")

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [18]:
submit = pd.read_csv("/Users/ming/Desktop/dsschool/titanic/gender_submission.csv", index_col = "PassengerId")

submit["Survived"] = predictions
#예제 파일에 예측 값 넣기

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [21]:
#저장하기
submit.to_csv("/Users/ming/Desktop/dsschool/titanic/decision_tree.csv")

In [20]:
# 78.947 나오면 된다. 오예오예